In [18]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 10 13:12:50 2020

@author: Cerys
"""

from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt

hdulist = fits.open('first_14dec17.fits')
data = hdulist[1].data
n = 100000

ra = np.empty(n)
for i in range(n):
    ra[i] = (data[i+200000])[0] #in degrees

#sources are given in orders of dec
dec = np.empty(n)
for i in range(n):
    dec[i] = (data[i+200000])[1] #in degrees

far_away = []
for i in range(n):
    source_ra = ra[i]
    source_dec = dec[i]
    ra_distances = ra - ra[i]
    ra_distances[i] = 10 
    dec_distances = dec - dec[i]
    dec_distances[i] = 10  
    total_distance = np.sqrt(ra_distances**2+dec_distances**2)
    nearest_neighbour = np.min(total_distance)
    if nearest_neighbour > 0.07:
        far_away.append(i)

In [ ]:
# if you need to plot the histogram
d = []
for m in range(n-1):
    dec_dist = dec - dec[m]
    ra_dist = ra - ra[m]
    distance = np.sqrt(dec_dist**2+ra_dist**2)
    distance[m] = 10*8
    nearest_neighbour = np.min(distance)
    d.append(nearest_neighbour)
    
plt.hist(d,bins=100)
plt.axvline(0.07)

In [19]:
fpeak = np.empty(n)
for i in range(n):
    fpeak[i] = (data[i+200000])[3]
    
fint = np.empty(n)
for i in range(n):
    fint[i] = (data[i+200000])[4]
    
major = np.empty(n)
for i in range(n):
    major[i] = (data[i+200000])[9]
   
minor = np.empty(n)
for i in range(n):
    minor[i] = (data[i+200000])[10]

In [20]:
e = []
ratios = []
for i in far_away:
    ratio = major[i]/minor[i]
    ratios.append(ratio)
    if 1.5<ratio <2:
        e.append(i)
        
print (len(far_away))
print (len(e))
print (np.max(ratios))

26713
3154
6.272912463183508


In [21]:
f = []
for i in e:
    if fpeak[i] > 3:
        f.append(i)
        
len(f)

429

In [16]:
for i in f:
    print (ra[i]/15,dec[i])

11.708707336899607 51.631373860796515
15.904543785615289 51.53850719374644
12.054731337926723 51.53665962757595
17.190771314512183 51.533654344341144
8.61005996959937 51.5330942688292
9.695241335622885 51.526612419306865
9.403241641685947 51.52617528719998
11.151668071717605 51.50960183317895
14.453068481322882 51.44955422509078
8.252437639500728 51.428800695297696
10.753105587733906 51.423906864757384
11.954635128155086 51.41447095841901
7.725682522406672 51.345906104435834
10.864422189442996 51.311464875546044
14.27147196513003 51.27942104305493
12.563275349309746 51.272474740669026
13.685557392212882 51.26756041956121
16.426499556452548 51.2571068150365
17.153455714346684 51.25082645609467
13.566522676741677 51.238624323142446
16.224577307981626 51.236500134310575
7.833550333930849 51.22053798222025
13.078966470640271 51.1885248855804
10.008745197142693 51.15999860051184
16.008371767918376 51.053215423027865
8.940455344282562 50.99211255071293
15.10976475995482 50.97896785165524
15.

In [22]:
from selenium import webdriver
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException
#Uses chrome driver - needs to be downloaded before running code
#The version of chrome driver used must be the same as the chrome browser
#Chrome driver can be used remotely once downloaded
driver = webdriver.Chrome()
#Open chrome and maximise window
driver.maximize_window()

#Get webpage that cutouts can be downloaded from
driver.get("https://third.ucllnl.org/cgi-bin/firstcutout")

#Assign required image size (in arcmin) by finding image size element and changing its value
size_arcmin = 2.5
imagesize = driver.find_element_by_name('ImageSize')
driver.execute_script("arguments[0].value=arguments[1]",imagesize,'{}'.format(size_arcmin))

#Loop to copy each RA and dec into the cutout search engine and download each image
for i in range(len(f)):
    try:
        #Finds search box and changes value to a string with RA and dec 
        radec = driver.find_element_by_name('RA')
        driver.execute_script("arguments[0].value=arguments[1]",radec,"{}\t+{}".format((ra[f[i]]/15),dec[f[i]]))
        #Clicks on button to select download type to be a FITS image
        fits_file = driver.find_elements_by_name('ImageType')[2]
        fits_file.click()
        #Clicks submit to download image
        submit = driver.find_element_by_name('.submit')
        submit.click()
    except NoSuchElementException:
        #Exception incase the input RA and dec are out of range of the database
        #Resets the webpage and continues loop through remaining values
        print ('Could not find element {}'.format(i))
        driver.get("https://third.ucllnl.org/cgi-bin/firstcutout")
        time.sleep(3)
        imagesize = driver.find_element_by_name('ImageSize') #in arcmins
        driver.execute_script("arguments[0].value=arguments[1]",imagesize,'2.5')
        continue

In [ ]:
ra[f]

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import os
from astropy import stats
from scipy.ndimage import rotate

directory_path = r'C:\Users\Cerys\Documents\Physics\Y4 Project\Semester 2\compact sources'
directory = os.fsencode(directory_path)
files = [f for f in os.listdir(directory_path) if f.endswith('.fits')]

n = 500 # number of objects to display
cols = 5

fig = plt.figure(figsize=(12.5,(n+cols-1)//cols*2.5))
for i in range(n):
    ax = fig.add_subplot((n+cols-1)//cols, cols, i+1)
    filename = os.fsdecode(files[i]) # could use if filename.endswith(".fits")
    filepath = os.path.join(directory_path, filename)# or #filepath = directory_path + '\\' + filename
    sigma=3
    data = fits.getdata(filepath, ext=0)
    data[np.isnan(data)] = 0
    _,median,std = stats.sigma_clipped_stats(data[:,:], sigma=sigma)
    data[data<median+sigma*std] = median+sigma*std
    ax.imshow(data, cmap='inferno')
    ax.text(10,10,'{}'.format(i),color='white')
    plt.axis('off')
fig.tight_layout(pad=0.1)
plt.show()